In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-03 09:29:26,850 INFO: Initializing external client
2025-03-03 09:29:26,851 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 09:29:28,036 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214677
Fetching data from 2025-02-02 14:29:26.846725+00:00 to 2025-03-03 13:29:26.846725+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (9.26s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-02 15:00:00+00:00,2,0
1,2025-02-02 16:00:00+00:00,2,0
2,2025-02-02 17:00:00+00:00,2,0
3,2025-02-02 18:00:00+00:00,2,0
4,2025-02-02 19:00:00+00:00,2,0
...,...,...,...
174440,2025-03-03 09:00:00+00:00,263,140
174441,2025-03-03 10:00:00+00:00,263,95
174442,2025-03-03 11:00:00+00:00,263,99
174443,2025-03-03 12:00:00+00:00,263,91


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174445 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         174445 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  174445 non-null  int32                  
 2   rides               174445 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174445 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174445 non-null  datetime64[us]
 1   pickup_location_id  174445 non-null  int32         
 2   rides               174445 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [9]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [10]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,2,0,0,0,3,0,0,3,5,1,...,0,0,3,0,0,1,0,0,255,2025-02-03 11:00:00
1,2,1,0,0,1,1,1,0,1,1,...,3,0,0,0,0,1,0,0,218,2025-02-21 16:00:00
2,369,273,249,21,251,348,17,373,286,251,...,250,275,1,222,32,324,393,166,237,2025-02-24 23:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,57,2025-02-11 00:00:00
4,0,0,0,0,0,0,0,1,0,1,...,0,0,0,1,0,1,0,1,56,2025-02-27 13:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,187,2025-02-28 12:00:00
247,0,2,0,0,4,0,1,0,0,2,...,0,3,1,2,0,0,0,2,71,2025-02-08 05:00:00
248,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,8,2025-02-08 08:00:00
249,0,1,1,1,1,2,0,4,0,1,...,1,1,0,1,1,2,4,1,168,2025-02-18 11:00:00


In [11]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-03 09:29:54,794 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 09:29:54,799 INFO: Initializing external client
2025-03-03 09:29:54,800 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 09:29:56,080 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214677
Fetching data from 2025-02-02 14:29:54.794365+00:00 to 2025-03-03 13:29:54.794365+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.23s) 


In [12]:
current_date

Timestamp('2025-03-03 14:29:54.794365+0000', tz='Etc/UTC')

In [13]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-02 15:00:00
1,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,0,0,0,3,2025-03-02 15:00:00
2,0,7,2,8,1,0,4,1,0,1,...,0,1,4,5,3,5,6,4,4,2025-03-02 15:00:00
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-02 15:00:00
4,6,2,0,0,3,1,2,1,3,2,...,1,1,2,3,1,2,1,2,7,2025-03-02 15:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,259,2025-03-02 15:00:00
247,0,0,1,4,0,2,0,0,1,1,...,0,0,0,0,2,1,1,1,260,2025-03-02 15:00:00
248,38,26,37,26,25,9,5,14,5,2,...,3,7,13,15,23,38,36,41,261,2025-03-02 15:00:00
249,75,59,61,46,32,24,22,10,7,2,...,25,50,74,103,121,89,87,87,262,2025-03-02 15:00:00


In [14]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-03 09:30:10,169 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 09:30:10,173 INFO: Initializing external client
2025-03-03 09:30:10,174 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 09:30:11,428 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214677


In [15]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [16]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,4.0
3,6,0.0
4,7,3.0
...,...,...
246,259,0.0
247,260,0.0
248,261,43.0
249,262,85.0


In [17]:
predictions.sort_values("predicted_demand", ascending=False).head(10)

,pickup_location_id,predicted_demand
122,132,370.0
224,237,286.0
223,236,285.0
128,138,256.0
150,161,253.0
174,186,224.0
226,239,217.0
132,142,209.0
40,43,171.0
221,234,170.0
